## import modules

In [21]:
from models.issues import Issues
from models import config
from models.transform import transform_issues
from models.repos import Repos
import pandas as pd
from zenhub import Zenhub
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import interactive

## extract repos and their id

In [2]:
repos = Repos('k8-proxy').get_repos()[['name', 'id']]
repos=repos.rename(columns={"id":"repo_id", "name": "repo"})

## extract issues and their id

In [3]:
issues = transform_issues()[['repo', 'title','number', 'id']]
issues = issues.rename(columns={"id":"issue_id"})

## merge issues to their respective repos

In [5]:
issue = issues.merge(repos, on='repo')
issue.shape

(288, 5)

## create a Zenhub connection

In [7]:
zh = Zenhub(config.ZENHUB_TOKEN)

## extract issue data from zenhub in batches of 100

In [8]:
issue['pipes'] = ""
for i in range(0, 100):
    issue.pipes[i] = zh.get_issue_data(repo_id=issue.repo_id[i], issue_number=issue.number[i])

In [10]:
for i in range(100, 200):
    issue.pipes[i] = zh.get_issue_data(repo_id=issue.repo_id[i], issue_number=issue.number[i])

In [11]:
for i in range(200, len(issue)):
    issue.pipes[i] = zh.get_issue_data(repo_id=issue.repo_id[i], issue_number=issue.number[i])

In [12]:
issue.head()

,repo,title,number,issue_id,repo_id,pipes
0,aws-jmeter-test-engine-v1,removing unnecessary files,37,717701441,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
1,aws-jmeter-test-engine-v1,Bzmconcurrent thread group,36,717700784,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
2,aws-jmeter-test-engine-v1,Move Transactions for status checks to separat...,35,717457683,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
3,aws-jmeter-test-engine-v1,Implement auto scaling to automatically alloca...,33,716320805,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."
4,aws-jmeter-test-engine-v1,s3 out file upload is optimised,31,715178800,298066702,"{'plus_ones': [], 'is_epic': False, 'pipelines..."


## get the status of an issue from zenhub pipelines

In [13]:
issue['status'] = ""
for i in range(0, len(issue)):
    if 'pipelines' in issue['pipes'][i]:
        issue['status'][i] = issue.pipes[i]['pipelines'][0]['name']
    else:
        issue['status'][i] = None
boards = issue.drop('pipes', 1)
boards = boards.rename(columns = {"repo": "project", "title":"summary", "number":"task"})

## boards 

In [14]:
boards.head(2)

,project,summary,task,issue_id,repo_id,status
0,aws-jmeter-test-engine-v1,removing unnecessary files,37,717701441,298066702,Peer Review
1,aws-jmeter-test-engine-v1,Bzmconcurrent thread group,36,717700784,298066702,Peer Review


## sample for k8-data-visualization

In [17]:
boards.query("project=='k8-data-visualization'").head()

,project,summary,task,issue_id,repo_id,status
34,k8-data-visualization,PR report enhancement. Closes #110,113,718361309,289667349,Peer Review
35,k8-data-visualization,Replicate All Notebooks in Jupyter to Google C...,112,718239991,289667349,Backlog
36,k8-data-visualization,Replicate All Notebooks in Jupyter to Google C...,111,718239351,289667349,Backlog
37,k8-data-visualization,Remove the displayed data under Fetch “PR from...,110,718235134,289667349,Peer Review
38,k8-data-visualization,Enhance Make_PowerPoint_Presentation.ipynb,108,715774605,289667349,In Progress


## widget to slice the boards per project

In [26]:
items = ['All']+sorted(boards['project'].unique().tolist())
def view(Project=''):
    if Project=="All": display(boards)
    display(boards[boards.project==Project].reset_index(drop=True))
w = widgets.Select(options=items)

In [27]:
interactive(view, Project=w)

interactive(children=(Select(description='Project', options=('All', 'Open-Source', 'aws-jmeter-test-engine-v1'…

## widget to slice the boards per status

In [44]:
stats = boards['status'].unique().tolist()
stat = [x for x in stats if x is not None]
items = ['All']+sorted(stat)
def view(Status=''):
    if Status=="All": display(boards)
    display(boards[boards.status==Status].reset_index(drop=True))
w = widgets.Select(options=items)

In [45]:
interactive(view, Status=w)

interactive(children=(Select(description='Status', options=('All', 'Backlog', 'Done', 'GW Team Review', 'In Pr…

In [46]:
# save to csv
boards.to_csv("../data/boards.csv", index=False)